# Notebook 2 - SQL avec vraies bases de données
## Analyse e-commerce avec PostgreSQL en ligne




### 🎯 Objectifs pédagogiques
- Connecter Python à une vraie base de données PostgreSQL
- Écrire des requêtes SQL complexes sur des données réelles
- Implémenter des analyses RFM avec SQL
- Intégrer SQL et pandas pour des analyses avancées
- Gérer les connexions et la sécurité

### 🛍️ Contexte du projet
Vous analysez les données d'un vrai dataset e-commerce (Brazilian E-Commerce Public Dataset) hébergé sur une base PostgreSQL.

Objectif : créer une segmentation clientèle pour optimiser les campagnes marketing.


## Partie 1 : Connexion à la base de données réelle

### 🔧 Installation et configuration


# Installation des dépendances


```
pip install psycopg2-binary sqlalchemy pandas python-dotenv
```




In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import os
from dotenv import load_dotenv

from sqlalchemy import create_engine, text

### 🌐 Base de données PostgreSQL gratuite (ElephantSQL)

**Option 1 : ElephantSQL (20MB gratuit)**
1. Créez un compte sur [elephantsql.com](https://www.elephantsql.com/)
2. Créez une instance "Tiny Turtle" (gratuite)
3. Récupérez vos credentials

**Option 2 : Supabase (500MB gratuit)**
1. Créez un compte sur [supabase.com](https://supabase.com/)
2. Créez un nouveau projet
3. Récupérez l'URL de connexion PostgreSQL

In [2]:
# Configuration de connexion (à adapter selon votre provider)
DATABASE_CONFIG = {
    'host': 'aws-0-eu-west-3.pooler.supabase.com',  # Ou votre host Supabase
    'database': os.getenv("DATABASE"),
    'user': os.getenv("USER_postgres"),
    'password': os.getenv("PASSWORD"),
    'port': os.getenv("PORT"),
}

# Création de l'engine SQLAlchemy
engine = create_engine(
    f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@"
    f"{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
)

# Test de connexion
def test_connection():
    """
    Testez votre connexion à la base

    Étapes :
    1. Utilisez pd.read_sql() pour exécuter "SELECT version()"
    2. Affichez la version PostgreSQL
    3. Gérez les erreurs de connexion
    """
    try:
        # Votre code ici
        version = pd.read_sql("SELECT version();", engine)
        print(version)
        pass
    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return False
    return True

test_connection()

                                             version
0  PostgreSQL 17.4 on aarch64-unknown-linux-gnu, ...


True


## Partie 2 : Import du dataset e-commerce

### 📊 Dataset Brazilian E-Commerce
Nous utilisons le célèbre dataset Olist (100k commandes réelles).

**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [3]:
### 📥 Import des données via API

import requests
import zipfile
import io
import pandas as pd

def download_olist_dataset():

    """
    Télécharge le dataset Olist depuis Kaggle API

    Alternative : Utilisez l'API publique de l'IBGE (Institut brésilien)
    pour des données e-commerce synthétiques mais réalistes
    """

    # URL des données publiques brésiliennes
    IBGE_API = "https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce"

    # Récupération des données de villes (pour la géolocalisation)
    cities_url = f"{IBGE_API}localidades/municipios"

    try:
        response = requests.get(cities_url)
        cities_data = response.json()

        # Convertir en DataFrame
        cities_df = pd.DataFrame(cities_data)

        # Votre code pour nettoyer et structurer
        # Créez des données e-commerce réalistes basées sur ces villes

        return cities_df
    except Exception as e:
        print(f"Erreur API IBGE : {e}")
        return None

# Génération de données e-commerce réalistes
def generate_ecommerce_data(cities_df, n_customers=10000):
    """
    Génère des données e-commerce réalistes

    Étapes guidées :
    1. Sélectionnez 50 villes brésiliennes aléatoirement
    2. Créez des clients avec distribution réaliste
    3. Générez des commandes avec saisonnalité
    4. Ajoutez des produits avec catégories cohérentes
    5. Calculez des prix et frais de port basés sur la distance
    """
    pass

1. customers : customer_id, customer_city, customer_state
2. orders : order_id, customer_id, order_status, order_date, order_delivered_date
3. order_items : order_id, product_id, seller_id, price, freight_value
4. products : product_id, product_category, product_weight_g
5. sellers : seller_id, seller_city, seller_state

In [4]:
customers_clean = (
    pd.read_csv("data/olist_customers_dataset.csv")
    .dropna(subset=['customer_id', 'customer_city', 'customer_state'])
    [['customer_id', 'customer_city', 'customer_state']]
    .drop_duplicates()
)

customers_clean['customer_id'] = customers_clean['customer_id'].astype(str)

print(customers_clean.head())

                        customer_id          customer_city customer_state
0  06b8999e2fba1a1fbc88172c00ba8bc7                 franca             SP
1  18955e83d337fd6b2def6b18a428ac77  sao bernardo do campo             SP
2  4e7b3e00288586ebd08712fdd0374a03              sao paulo             SP
3  b2b6027bc5c5109e529d4dc6358b12c3        mogi das cruzes             SP
4  4f2d8ab171c80ec8364f7c12e35b23ad               campinas             SP


In [5]:
orders_clean = (
    pd.read_csv("data/olist_orders_dataset.csv")
    .dropna(subset=['order_id', 'customer_id', 'order_purchase_timestamp', 'order_delivered_customer_date'])
    [['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_delivered_customer_date']]
    .rename(columns={
        'order_purchase_timestamp': 'order_date',
        'order_delivered_customer_date': 'order_delivered_date'
    })
    .drop_duplicates()
)

# Convert date columns
orders_clean['order_date'] = pd.to_datetime(orders_clean['order_date'])
orders_clean['order_delivered_date'] = pd.to_datetime(orders_clean['order_delivered_date'])

In [6]:
df_order_items = pd.read_csv("data/olist_order_items_dataset.csv")

order_items_clean = df_order_items[["order_id", "product_id", "seller_id", "price", "freight_value"]]

order_items_clean = order_items_clean.dropna(subset=["order_id", "product_id", "seller_id", "price", "freight_value"])

order_items_clean = order_items_clean.drop_duplicates(subset=["order_id", "product_id", "seller_id"])

print(order_items_clean.head())

                           order_id                        product_id  \
0  00010242fe8c5a6d1ba2dd792cb16214  4244733e06e7ecb4970a6e2683c13e61   
1  00018f77f2f0320c557190d7a144bdd3  e5f2d52b802189ee658865ca93d83a8f   
2  000229ec398224ef6ca0657da4fc703e  c777355d18b72b67abbeef9df44fd0fd   
3  00024acbcdf0a6daa1e931b038114c75  7634da152a4610f1595efa32f14722fc   
4  00042b26cf59d7ce69dfabb4e55b4fd9  ac6c3623068f30de03045865e4e10089   

                          seller_id   price  freight_value  
0  48436dade18ac8b2bce089ec2a041202   58.90          13.29  
1  dd7ddc04e1b6c2c614352b383efe2d36  239.90          19.93  
2  5b51032eddd242adc84c38acab88f23d  199.00          17.87  
3  9d7a1d34a5052409006425275ba1c2b4   12.99          12.79  
4  df560393f3a51e74553ab94004ba5c87  199.90          18.14  


In [7]:
products_clean = (
    pd.read_csv("data/olist_products_dataset.csv")
    .dropna(subset=['product_id', 'product_category_name', 'product_weight_g'])
    [['product_id', 'product_category_name', 'product_weight_g']]
    .rename(columns={'product_category_name': 'product_category'})
    .drop_duplicates()
)

print(products_clean.head())

                         product_id       product_category  product_weight_g
0  1e9e8ef04dbcff4541ed26657ea517e5             perfumaria             225.0
1  3aa071139cb16b67ca9e5dea641aaa2f                  artes            1000.0
2  96bd76ec8810374ed1b65e291975717f          esporte_lazer             154.0
3  cef67bcfe19066a932b7673e239eb23d                  bebes             371.0
4  9dc1a7de274444849c219cff195d0b71  utilidades_domesticas             625.0


In [8]:
sellers_clean = (
    pd.read_csv("data/olist_sellers_dataset.csv")
    .dropna(subset=['seller_id', 'seller_city', 'seller_state'])
    [['seller_id', 'seller_city', 'seller_state']]
    .drop_duplicates()
)

print(order_items_clean.head())


                           order_id                        product_id  \
0  00010242fe8c5a6d1ba2dd792cb16214  4244733e06e7ecb4970a6e2683c13e61   
1  00018f77f2f0320c557190d7a144bdd3  e5f2d52b802189ee658865ca93d83a8f   
2  000229ec398224ef6ca0657da4fc703e  c777355d18b72b67abbeef9df44fd0fd   
3  00024acbcdf0a6daa1e931b038114c75  7634da152a4610f1595efa32f14722fc   
4  00042b26cf59d7ce69dfabb4e55b4fd9  ac6c3623068f30de03045865e4e10089   

                          seller_id   price  freight_value  
0  48436dade18ac8b2bce089ec2a041202   58.90          13.29  
1  dd7ddc04e1b6c2c614352b383efe2d36  239.90          19.93  
2  5b51032eddd242adc84c38acab88f23d  199.00          17.87  
3  9d7a1d34a5052409006425275ba1c2b4   12.99          12.79  
4  df560393f3a51e74553ab94004ba5c87  199.90          18.14  


In [9]:
def create_tables(engine):
    from sqlalchemy import text

    create_customers = text("""
    CREATE TABLE IF NOT EXISTS customers (
        customer_id VARCHAR PRIMARY KEY,
        customer_city VARCHAR(100),
        customer_state CHAR(2)
    );
    """)

    create_sellers = text("""
    CREATE TABLE IF NOT EXISTS sellers (
        seller_id VARCHAR PRIMARY KEY,
        seller_city VARCHAR(100),
        seller_state CHAR(2)
    );
    """)

    create_products = text("""
    CREATE TABLE IF NOT EXISTS products (
        product_id VARCHAR PRIMARY KEY,
        product_category VARCHAR,
        product_weight_g NUMERIC
    );
    """)

    create_orders = text("""
    CREATE TABLE IF NOT EXISTS orders (
        order_id VARCHAR PRIMARY KEY,
        customer_id VARCHAR,
        order_status VARCHAR(50),
        order_date TIMESTAMP,
        order_delivered_date TIMESTAMP,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
    """)

    create_order_items = text("""
    CREATE TABLE IF NOT EXISTS order_items (
        order_id VARCHAR,
        product_id VARCHAR,
        seller_id VARCHAR,
        price NUMERIC,
        freight_value NUMERIC,
        PRIMARY KEY (order_id, product_id, seller_id),
        FOREIGN KEY (order_id) REFERENCES orders(order_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id),
        FOREIGN KEY (seller_id) REFERENCES sellers(seller_id)
    );
    """)

    with engine.connect() as conn:
        conn.execute(create_customers)
        conn.execute(create_sellers)
        conn.execute(create_products)
        conn.execute(create_orders)
        conn.execute(create_order_items)
        conn.commit()

In [10]:
create_tables(engine)

query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""

# Execute query
with engine.connect() as conn:
    tables_df = pd.read_sql(query, conn)

print(tables_df)

In [11]:
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""

# Execute query
with engine.connect() as conn:
    tables_df = pd.read_sql(query, conn)

print(tables_df)

    table_name
0    customers
1       orders
2  order_items
3     products
4      sellers


In [12]:
tables = ["customers", "orders", "products", "sellers", "order_items"]

# Exécution de la requête pour chaque table
with engine.connect() as conn:
    for table in tables:
        result = conn.execute(text(f"SELECT COUNT(*) FROM {table}"))
        count = result.scalar()
        print(f"{table} : {count} lignes")


customers : 0 lignes
orders : 0 lignes
products : 0 lignes
sellers : 0 lignes
order_items : 0 lignes


In [13]:
customers_clean.to_sql("customers", engine, if_exists="append", index=False)
print("Donnees inseres dans customers")

Donnees inseres dans customers


In [14]:
sellers_clean.to_sql("sellers", engine, if_exists="append", index=False)



95

In [15]:
products_clean.to_sql("products", engine, if_exists="append", index=False)


340

In [16]:
orders_clean.to_sql("orders", engine, if_exists="append", index=False)
print("Donnees inseres dans orders_clean")


Donnees inseres dans orders_clean


In [18]:
order_items_clean.to_sql("order_items", engine, if_exists="append", index=False)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "order_items" violates foreign key constraint "order_items_order_id_fkey"
DETAIL:  Key (order_id)=(002f19a65a2ddd70a090297872e6d64e) is not present in table "orders".

[SQL: INSERT INTO order_items (order_id, product_id, seller_id, price, freight_value) VALUES (%(order_id__0)s, %(product_id__0)s, %(seller_id__0)s, %(price__0)s, %(freight_value__0)s), (%(order_id__1)s, %(product_id__1)s, %(seller_id__1)s, %(price__1)s, %( ... 101185 characters truncated ... (%(order_id__999)s, %(product_id__999)s, %(seller_id__999)s, %(price__999)s, %(freight_value__999)s)]
[parameters: {'freight_value__0': 13.29, 'order_id__0': '00010242fe8c5a6d1ba2dd792cb16214', 'seller_id__0': '48436dade18ac8b2bce089ec2a041202', 'product_id__0': '4244733e06e7ecb4970a6e2683c13e61', 'price__0': 58.9, 'freight_value__1': 19.93, 'order_id__1': '00018f77f2f0320c557190d7a144bdd3', 'seller_id__1': 'dd7ddc04e1b6c2c614352b383efe2d36', 'product_id_

## Partie 3 : Requêtes SQL avancées


### 🔍 Analyses SQL à implémenter

#### 1. Analyse RFM (Récence, Fréquence, Montant)
```sql
-- Votre défi : Calculer les métriques RFM pour chaque client
WITH customer_metrics AS (
    SELECT
        c.customer_id,
        c.customer_state,
        -- Récence : jours depuis dernier achat
        -- Fréquence : nombre de commandes
        -- Montant : total dépensé
        
        -- Complétez cette requête CTE
        
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id, c.customer_state
)

-- Créez les segments RFM (Champions, Loyaux, À risque, etc.)
SELECT
    customer_id,
    customer_state,
    recency_score,
    frequency_score,
    monetary_score,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        -- Ajoutez les autres segments
        ELSE 'Others'
    END as customer_segment
FROM customer_metrics;
```

In [ ]:
#### 2. Analyse géographique des ventes

def geographic_sales_analysis():
    """
    Analysez les performances par état/région

    Requêtes à écrire :
    1. Top 10 des états par CA
    2. Croissance MoM par région
    3. Taux de conversion par ville
    4. Distance moyenne vendeur-acheteur
    """

    query_top_states = """
    -- Votre requête SQL ici
    -- Utilisez des JOINs et GROUP BY
    -- Calculez le CA, nombre de commandes, panier moyen
    """

    return pd.read_sql(query_top_states, engine)

#### 3. Analyse temporelle et saisonnalité
```sql
-- Détectez les patterns saisonniers
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
```

---

## Partie 4 : Analyse prédictive avec SQL

### 🔮 Modèles simples en SQL

In [ ]:
#### 1. Prédiction de churn

def churn_prediction_sql():
    """
    Identifiez les clients à risque de churn

    Indicateurs :
    - Pas d'achat depuis X jours
    - Baisse de fréquence d'achat
    - Diminution du panier moyen
    - Changement de comportement géographique
    """

    churn_query = """
    WITH customer_activity AS (
        -- Calculez les métriques d'activité récente
        -- Comparez avec l'historique du client
        -- Scorez le risque de churn
    )

    SELECT
        customer_id,
        days_since_last_order,
        order_frequency_trend,
        monetary_trend,
        churn_risk_score,
        CASE
            WHEN churn_risk_score > 0.7 THEN 'High Risk'
            WHEN churn_risk_score > 0.4 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as churn_segment
    FROM customer_activity;
    """

    return pd.read_sql(churn_query, engine)


#### 2. Recommandations produits
```sql
-- Market Basket Analysis simplifié
WITH product_pairs AS (
    SELECT
        oi1.product_id as product_a,
        oi2.product_id as product_b,
        COUNT(*) as co_purchase_count
    FROM order_items oi1
    JOIN order_items oi2 ON oi1.order_id = oi2.order_id
    WHERE oi1.product_id != oi2.product_id
    GROUP BY oi1.product_id, oi2.product_id
    HAVING COUNT(*) >= 10  -- Seuil minimum
)

SELECT
    product_a,
    product_b,
    co_purchase_count,
    co_purchase_count::float / total_a.count as confidence
FROM product_pairs pp
JOIN (
    SELECT product_id, COUNT(*) as count
    FROM order_items
    GROUP BY product_id
) total_a ON pp.product_a = total_a.product_id
ORDER BY confidence DESC;
```

---

## Partie 5 : Intégration avec les APIs météo

### 🌤️ Croisement données météo/ventes
```python
def weather_sales_correlation():
    """
    Correlez vos données météo du Notebook 1 avec les ventes
    
    Hypothèses à tester :
    1. Les ventes de certaines catégories augmentent-elles avec la pluie ?
    2. Y a-t-il un impact de la température sur les achats ?
    3. Les livraisons sont-elles impactées par la météo ?
    """
    
    # Récupérez les données météo historiques pour les villes brésiliennes
    weather_query = """
    SELECT DISTINCT customer_city, customer_state
    FROM customers
    WHERE customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC')
    ORDER BY customer_city;
    """
    
    cities = pd.read_sql(weather_query, engine)
    
    # Intégrez avec l'API météo
    # Analysez les corrélations
    
    pass
```

### 📊 Dashboard géo-temporel
```python
def create_geotemporal_dashboard():
    """
    Créez un dashboard interactif combinant :
    - Carte des ventes par région
    - Évolution temporelle avec météo
    - Segments clients géolocalisés
    - Prédictions par zone géographique
    """
    pass
```

---
## 🏆 Livrables finaux

### 📈 Rapport d'analyse complet
1. **Segmentation RFM (Recency, Frenquency, Monetary) ** : 5-7 segments avec caractéristiques
2. **Analyse géographique**  : Performances par région + recommandations
3. **Prédictions churn** : Liste des clients à risque + actions
4. **Recommandations produits** : Top 10 des associations
5. **Impact météo** : Corrélations significatives identifiées

### 🚀 Pipeline automatisé
```python
def automated_analysis_pipeline():
    """
    Pipeline qui :
    1. Se connecte à la DB
    2. Exécute toutes les analyses
    3. Met à jour les segments clients
    4. Génère le rapport automatiquement
    5. Envoie des alertes si nécessaire
    """
    pass
```

---

## 🎓 Auto-évaluation

- [ ] **Connexion DB** : PostgreSQL fonctionnelle
- [ ] **Requêtes complexes** : JOINs, CTEs, fonctions analytiques
- [ ] **Gestion des erreurs** : Connexions robustes
- [ ] **Performance** : Requêtes optimisées avec index
- [ ] **Intégration** : SQL + Python + APIs
- [ ] **Insights actionables** : Recommandations business claires

### 🔗 Préparation au Notebook 3
Le prochain notebook portera sur NoSQL (MongoDB) avec des données de réseaux sociaux et d'IoT, en temps réel.

### 💡 Bases de données alternatives
- **PlanetScale** : MySQL serverless gratuit
- **MongoDB Atlas** : 512MB gratuit
- **FaunaDB** : Base multi-modèle gratuite
- **Hasura Cloud** : GraphQL + PostgreSQL